In [19]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.stats import norm 
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , f1_score ,recall_score , precision_score,accuracy_score ,confusion_matrix ,roc_curve, auc, roc_auc_score
from sklearn.model_selection import GridSearchCV
#import mlxtend
#from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import DBSCAN
from sklearn import metrics

In [20]:
%matplotlib inline
hyper = pd.read_csv('export_CKD5-7-11AllFeatures.csv')
feature_names = ['age', 'bp', 'sg', 'al' , 'su' , 'rbc' , 'pc' , 'pcc','ba','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc','htn',
                'dm','cad','appet','pe','ane']
X = hyper[feature_names]
y = hyper['classification']



In [21]:
LR= LogisticRegression()
KNN= KNeighborsClassifier()
CART= DecisionTreeClassifier()
NB= GaussianNB()
SVM= SVC(kernel='linear',random_state=42)
MLP=MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=20,random_state=1)
DT=DecisionTreeClassifier(random_state=42)
RF=RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

In [22]:
model = PCA(n_components=7).fit(X)
X_pc = model.transform(X)

# number of components
n_pcs= model.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

initial_feature_names = ['age', 'bp', 'sg', 'al' , 'su' , 'rbc' , 'pc' , 'pcc','ba','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc','htn',
                'dm','cad','appet','pe','ane','classification']
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(dic.items())
print(df)

     0    1
0  PC0   wc
1  PC1  bgr
2  PC2   bu
3  PC3  age
4  PC4   bp
5  PC5  sod
6  PC6  pcv


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_pc,y,test_size=30,random_state=7)
def model(model,X_tr, X_te, y_tr, y_te):
    model.fit(X_tr,y_tr)
    predict=model.predict(X_te)
    mae = np.mean(abs(predict - y_te))
    print('MAE = %0.4f' % mae)
    confusion_matrix(y_te, predict)
    print('accuracy score',accuracy_score(y_te,predict)*100)
    fpr, tpr, thresholds = metrics.roc_curve(y_te, predict, pos_label=1)
    auc=metrics.auc(fpr, tpr)
    print('AUC',auc)
    print('Classification report \n')
    print(classification_report(y_te,predict))
    print("Train acc: ",accuracy_score(y_tr,model.predict(X_tr)))
    print("test acc: ",accuracy_score(y_te,predict))

In [24]:
from sklearn.model_selection import cross_val_score
def crossV(model,X,y):
    print("Cross Validation: ")
    accuracy = cross_val_score(model, X, y, scoring='accuracy', cv = 10).mean() * 100
    print("Accuracy : " , accuracy)
    f1 = cross_val_score( model, X, y, cv=10, scoring='f1_macro').mean() * 100
    print("f1 : " , f1)

    recall = cross_val_score( model, X, y, cv=10, scoring='recall_macro').mean() * 100
    print("recall : " , recall)

    precision = cross_val_score( model, X, y, cv=10, scoring='precision_macro').mean() * 100
    print("precision : " , precision)

In [25]:
print("Logistic Regression:")
crossV(LR,X,y)
model(LR,X_train, X_test, y_train, y_test)
print("Suport Vector Machine:")
crossV(SVM,X,y)
model(SVM,X_train, X_test, y_train, y_test)
print("MLP:")
crossV(MLP,X,y)
model(MLP,X_train, X_test, y_train, y_test)
print("Naive Bayes:")
crossV(NB,X,y)
model(NB,X_train, X_test, y_train, y_test)
print(" KNN:")
crossV(KNN,X,y)
model(KNN,X_train, X_test, y_train, y_test)
print("Decision Tree :")
crossV(DT,X,y)
model(DT,X_train, X_test, y_train, y_test)
print("Random forest  :")
crossV(RF,X,y)
model(RF,X_train, X_test, y_train, y_test)
print("CART   :")
crossV(CART,X,y)
model(CART,X_train, X_test, y_train, y_test)

Logistic Regression:
Cross Validation: 
Accuracy :  98.00000000000001
f1 :  97.8648634092124


/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be ch

recall :  98.00000000000001
precision :  97.81593406593407
MAE = 0.0667
accuracy score 93.33333333333333
AUC 0.9411764705882353
Classification report 

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       1.00      0.88      0.94        17

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.94      0.93      0.93        30

Train acc:  0.9351351351351351
test acc:  0.9333333333333333
Suport Vector Machine:
Cross Validation: 
Accuracy :  96.0
f1 :  95.71039565203566
recall :  95.73333333333332
precision :  96.05439291101054
MAE = 0.0667
accuracy score 93.33333333333333
AUC 0.9411764705882353
Classification report 

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       1.00      0.88      0.94        17

    accuracy                           0.93        30
   macr

/Users/duna/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


recall :  96.6
precision :  95.46826625386997
MAE = 0.2333
accuracy score 76.66666666666667
AUC 0.7941176470588236
Classification report 

              precision    recall  f1-score   support

           0       0.65      1.00      0.79        13
           1       1.00      0.59      0.74        17

    accuracy                           0.77        30
   macro avg       0.82      0.79      0.76        30
weighted avg       0.85      0.77      0.76        30

Train acc:  0.8648648648648649
test acc:  0.7666666666666667
 KNN:
Cross Validation: 
Accuracy :  73.5
f1 :  73.08813821757224
recall :  75.46666666666667
precision :  74.59709199768089
MAE = 0.2333
accuracy score 76.66666666666667
AUC 0.7760180995475114
Classification report 

              precision    recall  f1-score   support

           0       0.69      0.85      0.76        13
           1       0.86      0.71      0.77        17

    accuracy                           0.77        30
   macro avg       0.77      0.78    